In [ ]:
# sudo apt install wireguard-tools wireguard

In [ ]:
import requests
import sys
from socket import AF_INET, AF_INET6
import uuid
import json
from ipaddress import ip_network, ip_address
from configparser import ConfigParser
from pathlib import Path
import gi
gi.require_version('NM', '1.0')
from gi.repository import NM, GLib

In [ ]:
sys.path.append(str(Path().absolute().parent))

In [ ]:
from eduvpn.oauth2 import run_challenge
from eduvpn.variants import EDUVPN
from eduvpn.crypto import gen_code_verifier, gen_code_challenge

In [ ]:
nm_client = NM.Client.new(None)
main_loop = GLib.MainLoop()

In [ ]:
code_challenge_method = "S256"

In [ ]:
info_base_url = 'https://vpn-next.tuxed.net/'
info_url = info_base_url + '/.well-known/vpn-user-portal'
info = requests.get(info_url).json()['api']['http://eduvpn.org/api#3']

In [ ]:
api_endpoint = info['api_endpoint']
token_endpoint = info['token_endpoint']
authorization_endpoint = info['authorization_endpoint']
api_endpoint

In [ ]:
oauth = run_challenge(
        token_endpoint=token_endpoint,
        authorization_endpoint=authorization_endpoint,
        app_variant=EDUVPN,
    )

In [ ]:
info = oauth.get(api_endpoint + '/info').json()['info']

In [ ]:
profile_list = info['profile_list']

In [ ]:
profile_list

In [ ]:
key = !wg genkey
key

In [ ]:
public_key = ! echo "{key[0]}" | wg pubkey
public_key

In [ ]:
connect = oauth.post(api_endpoint + '/connect', data={'profile_id': 'default-wg', 'public_key': public_key[0]})

In [ ]:
assert(connect.status_code == 200)
assert(connect.headers['Content-Type'] == 'application/x-wireguard-profile')

In [ ]:
wireguard_config = connect.content.decode()
print(wireguard_config)

In [ ]:
config = ConfigParser()
config.read_string(wireguard_config)

In [ ]:
def disconnect():
    disconnect = oauth.post(api_endpoint + '/disconnect', data={'profile_id': 'default-wg'})
    assert(disconnect.status_code == 204 )

In [ ]:
ipv4s = []
ipv6s = []
for ip in config['Interface']['Address'].split(','):
    parsed = ip_network(ip.strip(), strict=False)
    if parsed.version == 4:
        ipv4s.append(NM.IPAddress(AF_INET, str(parsed.network_address), parsed.prefixlen))
    elif parsed.version == 6:
        ipv6s.append(NM.IPAddress(AF_INET6, str(parsed.network_address), parsed.prefixlen))

In [ ]:
dns4 = []
dns6 = []
for ip in config['Interface']['DNS'].split(','):
    parsed = ip_address(ip.strip())
    if parsed.version == 4:
        dns4.append(str(parsed))
    elif parsed.version == 6:
        dns6.append(str(parsed))

In [ ]:
profile = NM.SimpleConnection.new()
s_con = NM.SettingConnection.new()
s_con.set_property(NM.SETTING_CONNECTION_ID, "wireguard")
s_con.set_property(NM.SETTING_CONNECTION_TYPE, "wireguard")
s_con.set_property(NM.SETTING_CONNECTION_UUID, str(uuid.uuid4()))
s_con.set_property(NM.SETTING_CONNECTION_INTERFACE_NAME, "wireguard")

# https://lazka.github.io/pgi-docs/NM-1.0/classes/WireGuardPeer.html#NM.WireGuardPeer
peer = NM.WireGuardPeer.new()
peer.set_endpoint(config['Peer']['Endpoint'], allow_invalid=False)
peer.set_public_key(config['Peer']['PublicKey'], accept_invalid=False)
for ip in config['Peer']['AllowedIPs'].split(','):
    peer.append_allowed_ip(ip.strip(), accept_invalid=False)
 

s_ip4 = NM.SettingIP4Config.new()
s_ip6 = NM.SettingIP6Config.new()

for i in dns4:
    s_ip4.add_dns(i)

for i in dns6:
    s_ip6.add_dns(i)

s_ip4.set_property(NM.SETTING_IP_CONFIG_METHOD, "manual")
s_ip6.set_property(NM.SETTING_IP_CONFIG_METHOD, "manual")

for i in ipv4s:
    s_ip4.add_address(i)



for i in ipv6s:
    s_ip6.add_address(i)


# https://lazka.github.io/pgi-docs/NM-1.0/classes/SettingWireGuard.html
w_con = NM.SettingWireGuard.new()
w_con.append_peer(peer)
w_con.set_property(NM.SETTING_WIREGUARD_PRIVATE_KEY, key[0])

profile.add_setting(s_ip4)
profile.add_setting(s_ip6)
profile.add_setting(s_con)
profile.add_setting(w_con)

persistent = False

In [ ]:
def print_values(setting, key, value, flags, data):
    print("  %s.%s: %s" % (setting.get_name(), key, value))
profile.for_each_setting_value(print_values, None)

In [ ]:
def added_cb(client, result, data):
    try:
        client.add_connection_finish(result)
        print("The connection profile has been successfully added to NetworkManager.")
    except Exception as e:
        sys.stderr.write("Error: %s\n" % e)
    main_loop.quit()
nm_client.add_connection_async(profile, persistent, None, added_cb, None)    
main_loop.run()

In [ ]:
# get the connection object
connection = [i for i in nm_client.get_connections() if i.get_id() == 'wireguard'][0]

In [ ]:
def connected_cb(client, result, data):
    try:
        print("connected")
        nm_client.activate_connection_finish(result)
    except Exception as e:
        sys.stderr.write("Error: %s\n" % e)
    main_loop.quit()
nm_client.activate_connection_async(connection, None, None, None, connected_cb, "bla")    
main_loop.run()